### Load Documents

In [1]:
import glob


contents = []
file_paths = glob.glob("documents/*.txt")
for file_path in file_paths:
    with open(file_path, "r", encoding='latin-1') as f:
        contents.append(f.read())

In [2]:
contents

["Quelle est l'age limite pour pouvoir faire le concours?La limite d\x92âge est de 22 ans au premier octobre 2021 et tout dossier irrecevable sera classé sans suite et sans remboursement des frais déjà payés.\nQuels sont les eleves qui peuvent se presenter au concours?Le concours est ouvert aux élèves des classes de terminales scientifiques ou techniques (S1, S2, S3, T1, T2), mais également aux bacheliers et candidats libres aux baccalauréats (S1, S2, S3, T1, T2).\nLes candidats seront évalués sur quelles matieres?Les matières à traiter sous forme de questions à choix multiples (QCM) sont : Mathématiques, Physique, Français, Anglais. La durée des épreuves est de trois heures.\nOù puis-je trouver des epreuves des années précédentes?Vous pouvez télécharger des épreuves à partir de l\x92onglet anciennes épreuves\nQuel est le delai maximal pour envoyer mon relevé?Pour les candidats des classes de terminale, en cas de réussite au bac, ils sont priés d\x92envoyer la photocopie légalisée du r

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 375,
    chunk_overlap = 20,
    length_function = len,
    separators= [
        "\n",
        "\n\n",
        " ",
        ".",
        ",",
        "\u200b",
        "\uff0c",
        "\u3001",
        "\uff0e",
        "\u3002",
        ""
    ]
)

In [4]:
all_docs = text_splitter.create_documents(contents)

In [11]:
all_docs[:5]

[Document(page_content="Quelle est l'age limite pour pouvoir faire le concours?La limite d\x92âge est de 22 ans au premier octobre 2021 et tout dossier irrecevable sera classé sans suite et sans remboursement des frais déjà payés."),
 Document(page_content='Quels sont les eleves qui peuvent se presenter au concours?Le concours est ouvert aux élèves des classes de terminales scientifiques ou techniques (S1, S2, S3, T1, T2), mais également aux bacheliers et candidats libres aux baccalauréats (S1, S2, S3, T1, T2).'),
 Document(page_content='Les candidats seront évalués sur quelles matieres?Les matières à traiter sous forme de questions à choix multiples (QCM) sont : Mathématiques, Physique, Français, Anglais. La durée des épreuves est de trois heures.\nOù puis-je trouver des epreuves des années précédentes?Vous pouvez télécharger des épreuves à partir de l\x92onglet anciennes épreuves'),
 Document(page_content='Quel est le delai maximal pour envoyer mon relevé?Pour les candidats des class

### Enrichment with keywords and questions_answered

In [6]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
import time
import warnings

warnings.filterwarnings("ignore")


llm = ChatOpenAI(
    openai_api_base = "https://api.groq.com/openai/v1",
    model="llama3-70b-8192",
    temperature=0.7,
    api_key="YOUR_KEY"
)

In [7]:
keyword_temp = """
<|start_header_id|>system<|end_header_id|>
Given a text as input determine its keywords.
Keywords are the most important words or group of words in reference to the given text, they give an overview of what is being adressed from it.
Only extract three keywords while making sure they are the most relevant ones.
You response should only consist of those three keywords do add any text apart from it
This is very important never give more than three keywords and only answer in French.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Text: {text}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Keywords:
"""
keyword_prompt = PromptTemplate.from_template(keyword_temp)

keyword_chain = (
    {'text': RunnablePassthrough()}
    | keyword_prompt
    | llm
    | StrOutputParser()
)

questions_temp = """
<|start_header_id|>system<|end_header_id|>
Given a text as input determine the questions that it could answer to. Extract exactly three questions while making sure they are the most relevant ones.
You response should only consist of those three questions do add any text apart from it
This is very important never give more than three keywords and only answer in French.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Text: {text}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Keywords:
"""
questions_prompt = PromptTemplate.from_template(questions_temp)

questions_chain = (
    {'text': RunnablePassthrough()}
    | questions_prompt
    | llm
    | StrOutputParser()
)

In [8]:
final_docs = []

start_idx = 0
for end_idx in range(0, len(all_docs), 30)[1:-1]:
    for doc in all_docs[start_idx:end_idx]:
        questions = questions_chain.invoke(doc.page_content)
        keywords = keyword_chain.invoke(doc.page_content)
        final_docs.append(
            Document(
                page_content=doc.page_content,
                metadata = {
                    "keywords": keywords,
                    "questions_answered": questions
                }
            )
        )
    start_idx = end_idx
    time.sleep(60)

for doc in all_docs[end_idx:len(all_docs)]:
        questions = questions_chain.invoke(doc.page_content)
        keywords = keyword_chain.invoke(doc.page_content)
        final_docs.append(
            Document(
                page_content=doc.page_content,
                metadata = {
                    "keywords": keywords,
                    "questions_answered": questions
                }
            )
        )

In [10]:
final_docs[:5]

[Document(page_content="Quelle est l'age limite pour pouvoir faire le concours?La limite d\x92âge est de 22 ans au premier octobre 2021 et tout dossier irrecevable sera classé sans suite et sans remboursement des frais déjà payés.", metadata={'keywords': 'Concours, âge, limite', 'questions_answered': "Quel est l'âge limite pour passer le concours ?\nQuel est le délai pour déposer son dossier ?\nQu'est-ce qui se passe si je dépasse l'âge limite ?"}),
 Document(page_content='Quels sont les eleves qui peuvent se presenter au concours?Le concours est ouvert aux élèves des classes de terminales scientifiques ou techniques (S1, S2, S3, T1, T2), mais également aux bacheliers et candidats libres aux baccalauréats (S1, S2, S3, T1, T2).', metadata={'keywords': 'Concours, Élèves, Baccalauréat', 'questions_answered': 'Qui peut se presenter au concours ?\nQuels sont les élèves éligibles pour participer au concours ?\nQuels sont les diplômes exigés pour participer au concours ?'}),
 Document(page_co

### Create And Store Embeddings

In [13]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embeddings_function = HuggingFaceEmbeddings(model_name="sentence-transformers/gtr-t5-large")
#db = Chroma.from_documents(final_docs, embeddings_function, persist_directory="./chroma_db")

In [20]:
db2 = Chroma(persist_directory="./chroma_db", embedding_function=embeddings_function)

### Create retriever with Reranking

In [17]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

retriever=db2.as_retriever()

In [19]:
retriever.get_relevant_documents("Génie Informatique")

[Document(page_content='dans la phase de conception que dans la mise en \x9cuvre des enseignements-apprentissages.La formation en Génie Informatique et Télécommunications (GIT) est dispensée depuis l\x92année académique 2012-2013 et est portée, au sein de l\x92EPT, par le département du Génie Informatique et Télécommunications (GIT). Elle a pour objectif de former des ingénieurs de conception', metadata={'keywords': 'Génie Informatique, Télécommunications,Formation', 'questions_answered': "Quel est le domaine d'étude de la formation en GIT ?\nQuel est l'objectif de la formation en Génie Informatique et Télécommunications ?\nDepuis quand la formation en GIT est-elle dispensée ?"}),
 Document(page_content='Initialement prévu parmi les axes du plan stratégique de l\x92EPT à la suite de l\x92autonomisation de celle-ci en 2009, le département du Génie Informatique et Télécommunications (GIT) fut créé dans la foulée par arrêté du Directeur de l\x92EPT suite à une proposition du conseil pédag

### Build the retrieval QA with context chain

In [20]:
from typing import Dict
from langchain.chains.combine_documents.base import DEFAULT_DOCUMENT_PROMPT
from langchain_core.prompts import format_document
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableBranch
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}

def format_docs(inputs: dict) -> str:
  inputs["context"] = "\n\n".join(
      f"Document {i}:\n {format_document(doc, DEFAULT_DOCUMENT_PROMPT)}" for i, doc in enumerate(inputs["context"])
  )
  return inputs

def format_chat_history(data):
  formatize_chat_history = ""
  if "chat_history" in data.keys():
    for message in data["chat_history"]:
      message_type = str(type(message)).split("'")[1].split(".")[-1]
      message_content = message.content.replace("\n", "")
      formatize_chat_history += f"\t{message_type}: {message_content}\n"
    data["chat_history"] = formatize_chat_history
  return data

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [21]:
# QA Prompt
qa_system_prompt = """
Je suis un assistant intelligent capable de discuter avec les utilisateur et de fournir des informations sur l'Ecole Polytechnique de Thiès communément appelé EPT. Sur la base du contexte et de la question de l'utilisateur, je fournis des informations à l'utilisateur de manière précise qui satisfont son besoin.

contexte: {context}
"""

qa_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "Je suis PolyBot une solution de chatbot basée sur l'IA Générative."),
    ("system", qa_system_prompt),
    ("human", "{input}"),
  ]
)

In [22]:
# Contextualization
contextualize_query_system_prompt = """
Je suis un assistant IA très util pour contextualiser les fils de discusssion. Compte tenu de l'historique du chat et de la 
dernière question de l'utilisateur, formule une question autonome qui peut être compris sans l'historique du chat.
Donne juste la question directe et brute qui sera la proche possible de la dernière question de l'utilisateur.
NE REPONDEZ PAS A LA QUESTION, reformulez-la au besoin ou retournez-la tel quel sinon.
Historique_Chat: 
{chat_history}

Dernière_Question: {input}
"""

contextualize_query_prompt = ChatPromptTemplate.from_template(contextualize_query_system_prompt)

In [23]:
contextualization_chain = RunnableBranch(
  (
    lambda x: not x.get("chat_history", False),
    lambda x: x["input"],
  ),
  format_chat_history | contextualize_query_prompt | llm | StrOutputParser(),
)

retrieval_chain = {"context": retriever , "input": RunnablePassthrough()}

chain = contextualization_chain | retrieval_chain | qa_prompt | llm | StrOutputParser()

In [27]:
chain.invoke(
    {"input": "Quel est la procédure pour candidater?"},
    config={
          "configurable": {"session_id": 'mass_sene'}
    }
)

"Pour candidater au MiTMN, vous devez fournir un dossier de candidature comprenant les pièces justificatives suivantes :\n\n* Extrait d'acte de naissance de moins de 3 mois\n* Copie légalisée de la pièce d'identité officielle\n* Copie légalisée du Baccalauréat\n* Copie légalisée du relevé de notes du Baccalauréat\n\nIl est important de vérifier que votre dossier est complet et que toutes les pièces sont légalisées pour éviter tout problème lors de l'inscription."

In [26]:
chain.get_graph().print_ascii()

            +---------------------------+          
            | format_chat_history_input |          
            +---------------------------+          
                           *                       
                           *                       
                           *                       
                      +--------+                   
                      | Branch |                   
                      +--------+                   
                           *                       
                           *                       
                           *                       
           +------------------------------+        
           | Parallel<context,input>Input |        
           +------------------------------+        
                  ***            ***               
                **                  **             
              **                      **           
+----------------------+          +-------------+  
| VectorStor